<a href="https://colab.research.google.com/github/hannaa-n/CS-491/blob/main/Medical_Questions_Nguyen_Nguyen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  NLP Assignment: Fine-Tuning a Transformer on Medical Question Pairs

In this assignment, you will fine-tune a transformer model to classify whether pairs of medical questions are paraphrases of each other.

Once you have everyting in place then before training, you should restart and change the runtime to TPU.

In [1]:
# Install necessary libraries
!pip install transformers datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [ ]:
##TODO: set random seed to your BlugoldID
seed=7835936

## Step 1: Load the Dataset
Use the `datasets` library to load the [curaihealth medical questions pairs](https://huggingface.co/datasets/curaihealth/medical_questions_pairs) dataset.

In [ ]:
from datasets import load_dataset
# TODO: Load the dataset
dataset = ####
dataset

##  Train/Validation/Test Split
The `medical_questions_pairs` dataset only provides a single training set. You need to create your own train, validation, and test sets.

We'll split the dataset into:
- **Train:** 80%
- **Validation:** 10%
- **Test:** 10%

Use `train_test_split` from the `datasets` library to do this.

In [ ]:
from datasets import DatasetDict

# Step 1: Split into train + temp (val + test)
temp_split = dataset['train'].train_test_split(test_size=0.2, seed=seed)

# Step 2: Split temp into validation + test (50/50 of temp = 10% each)
val_test_split = temp_split['test'].train_test_split(test_size=0.5, seed=seed)

# Step 3: Combine splits into a DatasetDict
split_dataset = DatasetDict({
    'train': temp_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test']
})

split_dataset

In [ ]:
## TODO: find an index  so that the question pair has label 0
idx_0=##
split_dataset['train'][idx_0]


In [ ]:
## TODO: find an index so that the question pair has label 1
idx_1=##
split_dataset['train'][idx_1]

## Step 2: Explore and Preprocess
Examine the fields. Tokenize question pairs using a pretrained tokenizer.

In [ ]:
from transformers import AutoTokenizer

 ##Choose a model checkpoint
checkpoint = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Tokenization function
def tokenize_fn(example):
    return tokenizer(example['question_1'], example['question_2'], truncation=True, padding='max_length',max_length=256)

##Apply to dataset
tokenized = split_dataset.map(tokenize_fn, batched=True)
tokenized

## Step 3: Load Model
Load a model for sequence classification.

In [ ]:
from transformers import AutoModelForSequenceClassification

# TODO: Define the model with correct number of labels
model = ##

## Step 4: Define Training Arguments
Use Hugging Face `TrainingArguments` to configure training.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none"
)

## Step 5: Define Trainer
Set up the `Trainer` object and begin training.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [ ]:
from transformers import Trainer

##Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Start training
##REMEMBER to set runtime type to GPU!
trainer.train()

## Step 6: Evaluation
Evaluate and inspect results.

In [ ]:
##Evaluate the model
metrics = trainer.evaluate()
print(metrics)

In [ ]:
# Save locally first
model.save_pretrained("medical-question-model")
tokenizer.save_pretrained("medical-question-model")

## Push to hub
## TODO: replace (your-username) with your huggingface user name
model.push_to_hub("your-username/medical-question-model")
tokenizer.push_to_hub("your-username/medical-question-model")

 ### Training Accuracy


Now that training is complete, let's evaluate the model on the training set to report training accuracy.

In [ ]:
# Evaluate on training data
train_metrics = trainer.evaluate(tokenized["train"])
print(f"Training Accuracy: {train_metrics['eval_accuracy']:.4f}")

### Evaluate Custom Question Pairs

Use this section to test your fine-tuned model on your own question pairs. This is useful for exploring how well the model generalizes to new examples outside the original dataset.

In [ ]:
#TODO: replace (your-username) with your huggingface user name
model_id = "your-username/medical-question-model"

The code block below sets up a function to use your fine-tuned model on a pair of text fields.

In [ ]:
import torch
import torch.nn.functional as F


def evaluate_question_pair(question1, question2, model_id=model_id):
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)
    model.eval()

    # Determine max length
    max_len = getattr(model.config, "max_position_embeddings", 512)

    # Tokenize with explicit max_length
    inputs = tokenizer(
        question1, question2,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_len
    )

    # Run through model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)

    predicted_class = torch.argmax(probs).item()
    confidence = probs[0][predicted_class].item()
    label_name = model.config.id2label.get(predicted_class, str(predicted_class))

    print(f"Q1: {question1}")
    print(f"Q2: {question2}")
    print(f"Predicted Label: {label_name} (Confidence: {confidence:.4f})")

    return predicted_class, confidence


In [ ]:
##Example usage:

evaluate_question_pair(
    "What are the symptoms of anemia?",
    "Can being tired all the time mean I have anemia?"
)

In [ ]:
##Use seed to select two integers between 1 and 100 that are not equal
##Reminder: seed is set above to be your Blugold ID

import random

# Use seed to select two integers between 1 and 100 that are not equal
random.seed(seed)

pick_1 = random.randint(1, 100)
pick_2 = random.randint(1, 100)

while pick_1 == pick_2:
  pick_2 = random.randint(1, 100)

print([pick_1, pick_2])


[82, 15]




---

### Additional Question Set

You can access a curated set of 100 medically-related questions here:  
[📄 Medical_Questions.csv](https://github.com/alex-smith-uwec/NLP_Spring2025/blob/main/Medical_Questions.csv)

These questions were **not part of the original training dataset**. They have been **synthetically generated** with care to ensure that they are medically relevant while **avoiding content that is gross, graphic, or sexual in nature**.

You will use this file to generate rephrasings in the next part of the assignment.

---

###  Generate and Evaluate Rephrasings
Look up the two rows in the spreadsheet file corresponding to your `pick_1` and `pick_2` to see which two questions are assigned to you.

You’ve already selected two questions, `pick_1` and `pick_2`, from the dataset above. In this part of the assignment, you will create **two new rephrasings** for each of these selected questions:

- **One rephrasing that preserves the original meaning**  
- **One rephrasing that alters the original meaning**

Then, you'll use your fine-tuned model to evaluate the similarity between the original and the rephrased versions.

 **Steps:**

1. For each of `pick_1` and `pick_2`, write:
   - One **meaning-preserving** rephrasing
   - One **meaning-changing** rephrasing

2. Use `evaluate_question_pair()` to compare:
   - `pick_1` with its two rephrasings
   - `pick_2` with its two rephrasings

This means you'll make **four total calls** to `evaluate_question_pair`.

Reflect on the model's behavior:
- Does it correctly identify the rephrasing that preserves meaning?
- Does it detect that the other rephrasing changes the intent?

Show the four calls below in code blocks. Then add a text block for the reflection on model behavior.